## Setup

### Loading
We first import `nbtschematic` to be able to load `.schematic` files. It uses `nbtlib` to parse the nbt file and some classes inherented from `numpy` classes to store the data. 

In [1]:
from nbtschematic import SchematicFile
sf = SchematicFile.load("schematics/apple.schematic")
sf

<SchematicFile 'Schematic': Schematic({'Blocks': ByteArray([Byte(0), Byte(-97), Byte(0), Byte(-97), Byte(-97), Byte(-97), Byte(0), Byte(-97), Byte(0), Byte(-97), Byte(-97), Byte(-97), Byte(-97), Byte(-97), Byte(-97), Byte(-97), Byte(-97), Byte(-97), Byte(0), Byte(-97), Byte(0), Byte(-97), Byte(-97), Byte(-97), Byte(0), Byte(-97), Byte(0), Byte(0), Byte(0), Byte(0), Byte(0), Byte(35), Byte(0), Byte(0), Byte(0), Byte(0)]), 'Materials': String('Alpha'), 'Data': ByteArray([Byte(0), Byte(14), Byte(0), Byte(14), Byte(14), Byte(14), Byte(0), Byte(14), Byte(0), Byte(14), Byte(14), Byte(14), Byte(14), Byte(14), Byte(14), Byte(14), Byte(14), Byte(14), Byte(0), Byte(14), Byte(0), Byte(14), Byte(14), Byte(14), Byte(0), Byte(14), Byte(0), Byte(0), Byte(0), Byte(0), Byte(0), Byte(13), Byte(0), Byte(0), Byte(0), Byte(0)]), 'TileEntities': List[BlockEntity]([]), 'Entities': List[Entity]([]), 'Length': Short(3), 'WEOffsetX': Int(0), 'WEOffsetY': Int(-2), 'WEOriginZ': Int(26), 'WEOffsetZ': Int(-2), 'Hei

### Parsing
What matters to us is the `blocks` property of the `SchematicFile` object. It is a 3D array of `Block` objects. Each `Block` object is a Byte type holding the block id. To be able to use it in our model, we need to convert it into a numpy `np.array`.

In [2]:
import numpy as np

np.asarray(sf.blocks)

array([[[  0, -97,   0],
        [-97, -97, -97],
        [  0, -97,   0]],

       [[-97, -97, -97],
        [-97, -97, -97],
        [-97, -97, -97]],

       [[  0, -97,   0],
        [-97, -97, -97],
        [  0, -97,   0]],

       [[  0,   0,   0],
        [  0,  35,   0],
        [  0,   0,   0]]], dtype=int8)

## First model - cube with 9^3 blocks and 16 block types

The first model is a simple cube with 9^3 blocks and 16 block types. 

**Objective:** Get the model to classify the blocks correctly, we will only provide schematics of cubes with ONLY 1 block type, but in random positions (We will basically fill the cube with 1 block type, and then remove some blocks randomly).

**Why:** This is to make sure we are able to train the model to classify the blocks correctly, then we will move on to more complex models where it will actually be classifying multiple block structures.

### Valid blocks

We will only be using the following blocks:
**Air, Dirt, Oak Log, Oak Leaves, Stone Brick, Cobblestone, Glass, Sandstone, Redstone Lamp, Iron Bars, Stone Brick, Bricks, Block of Quartz, White Wool, Bookshelf, White Terracotta, Nether Brick**


I already have a schematic file with all these blocks lined up in a row, so we will just load that and use it to get the block ids.

In [3]:
import pandas as pd

all_blocks_schematic = SchematicFile.load("schematics/allblocks.schematic")

# We reverse the array because I want the air block to be in the front, it doesn't really matter though
blocks = np.asarray(all_blocks_schematic.blocks).flatten()[::-1]

# Labels are in the same order as the blocks
labels = np.array(['Air', 'Dirt', 'Oak Log', 'Oak Leaves', 'Stone Brick', 'Cobblestone', 'Glass', 'Sandstone', 'Redstone Lamp', 'Iron Bars', 'Bricks', 'Block of Quartz', 'White Wool', 'Bookshelf', 'White Terracotta', 'Nether Brick'])

# Create a table with the labels (block names), block ids and index in the array
table = np.array([labels, blocks, np.arange(len(labels))]).T

# print the table using pandas
pd.DataFrame(table, columns=['Label', 'Block ID', 'Index']).style.hide()

Label,Block ID,Index
Air,0,0
Dirt,3,1
Oak Log,17,2
Oak Leaves,18,3
Stone Brick,97,4
Cobblestone,4,5
Glass,20,6
Sandstone,24,7
Redstone Lamp,123,8
Iron Bars,101,9


### One Hot Encoding the blocks

We will use a one hot encoding to represent the blocks. We will have a 16 length vector, with each index representing a block type. The index of the block type will be set to 1, and the rest will be 0.

**Why:** Block types is a categorical variable, and we need to represent it in a way that the model can understand. One hot encoding is a good way to do this. If we were to use a simple integer encoding, the model would think that the block types are ordinal, and that the block type with the highest integer is the best block type. This is not the case, so we use one hot encoding.

### Converting the blocks to one hot encoding

We will use the `block_ids` array to convert the blocks to one hot encoding. We will define a function `one_hot` to create a one hot encoded array and a `convert_blocks_to_one_hot` that takes an id an a dictionnary mapping the block ids to the one hot encoding index.

In [4]:
def one_hot(lenght: int, index: int)->np.array:
    """
    Creates a one hot array of the given lenght and sets the index to 1
    """
    one_hot = np.zeros((lenght))
    one_hot[index] = 1
    return one_hot

def convert_blocks_to_one_hot(block_id: int, possible_blocks: dict)->np.array:
    """
    Converts an array of block ids to a one hot array
    """
    return one_hot(len(possible_blocks), possible_blocks.get(block_id) or 0)

# We need to convert the block ids array to a dictionary of block ids and their index in the array
blocks_dict:dict = {block_id: index for index, block_id in enumerate(blocks)}

# convert_blocks_to_one_hot(3, blocks_dict)
one_hot(16, 0)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Creating the dataset

For other models, the dataset will be created inside Minecraft and exported as a `.schematic` file. For this model, we will create the dataset in python. We will randomly set blocks in the cube to a block type with the `random` module. Remember, in this model, **we are only training the model to classify the blocks correctly**, so we will only use 1 block type per cube. This should be a simple task.

In [5]:
from random import random, randint
from typing import List

dataset = []

class RandomCube:
    def __init__(self, block_index: int, dimensions: int=3, air_frequency=0.25):
        self.block_index = block_index
        self.dimensions = dimensions
        self.air_frequency = air_frequency
        self.data:np.array = []
        self.target:np.array = []

        self.create()

    def create(self):
        for x in range(self.dimensions):
            self.data.append([])
            for y in range(self.dimensions):
                self.data[x].append([])
                for z in range(self.dimensions):
                    if random() > self.air_frequency:
                        self.data[x][y].append(one_hot(16, self.block_index))
                    else:
                        self.data[x][y].append(one_hot(16, 0))

        self.data = np.array(self.data)
        self.target = one_hot(16, self.block_index)

        return self
    
    @property
    def count(self: int)->int:
        # Count each block and return a dict
        block_count = {}

        for x in self.data:
            for y in x:
               for z in y:
                   for i, is_there in enumerate(z):
                        if is_there == 1:
                            block_count[i] = block_count.get(i, 0) + 1

        return block_count
                           
    def __repr__(self):
        return f"RandomCube{{count={self.count}, data={self.data}, target={self.target}}}"
    
    def __str__(self):
        return f"RandomCube{{count={self.count}, data={self.data}, target={self.target}}}"
    

RandomCube(block_index=1).count

{0: 9, 1: 18}

Now that we have this RandomCube class, we can create a dataset about 70000 cubes. We will use 50000 for training, 20000 for testing.

In [6]:
from random import randint

def generate_dataset(size: int=1000, dimensions: int=3, air_frequency: float=0.25)->List[RandomCube]:
    dataset = []

    for _ in range(size):
        dataset.append(RandomCube(block_index=randint(0, 15), dimensions=dimensions, air_frequency=air_frequency))

    return dataset

dataset = generate_dataset(size=70000, dimensions=3, air_frequency=0.25)
print(f"Generated {len(dataset)} cubes, each cube has a size of {dataset[0].data.shape} and a target of {dataset[0].target.shape}")

Generated 70000 cubes, each cube has a size of (3, 3, 3, 16) and a target of (16,)


### Creating the model

It is a 4D model as we have 4 dimensions: x, y, z, and block type. For this simple model, we will use a simple NN with 2 hidden layers. We will use the `relu` activation function for the hidden layers, and `softmax` for the output layer. We will use the `categorical_crossentropy` loss function, and the `adam` optimizer.

In [13]:
import torch
import torch.nn as nn

class BlockClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(432, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
        )

    def forward(self, x):
        x = self.flatten(x)
        x = x.flatten()
        logits = self.linear_relu_stack(x)
        return logits
    

### Predicting the blocks

Let's create a random tensor and pass it through the model to see what it predicts. It will give a random result as the model is not trained yet. The weights are initialized randomly.

In [45]:
model_1 = BlockClassifier()
random_prediction = torch.rand(3,3,3,16)
prediction = model_1(random_prediction)

prediction = torch.sort(prediction, descending=True)

for index, value in enumerate(prediction.indices):
    print(f"{index+1}. {labels[value]} ~ {prediction.values[index] * 100:.2f}%")

1. Oak Log ~ 14.11%
2. Bricks ~ 10.61%
3. Stone Brick ~ 9.70%
4. Cobblestone ~ 9.08%
5. Sandstone ~ 8.79%
6. Air ~ 6.60%
7. White Terracotta ~ 5.56%
8. Nether Brick ~ 4.40%
9. Redstone Lamp ~ 3.83%
10. White Wool ~ 3.30%
11. Dirt ~ 2.29%
12. Iron Bars ~ -2.69%
13. Block of Quartz ~ -8.20%
14. Oak Leaves ~ -8.99%
15. Bookshelf ~ -9.55%
16. Glass ~ -9.89%


### Training the model

We will train the model for 10 epochs, with a batch size of 32. We will use the `ModelCheckpoint` callback to save the model after each epoch. We will also use the `EarlyStopping` callback to stop the training if the validation loss does not improve for 3 epochs.

In [46]:
def train(dataset, model, loss_fn, optimizer):
    size = len(dataset)

    for batch, cube in enumerate(dataset):
        # Compute prediction and loss
        pred = model(cube.data)
        loss = loss_fn(pred, cube.target)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(cube.data)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


# training_dataset = dataset[:50000]
#  = dataset[50000:]


